Imports

In [1]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time
import numpy as np

In [2]:
URL = {
    'BASE': 'https://{proxy}.api.pvp.net/api/lol/{region}/v2.2/{rest}',
    'STATIC_BASE': 'https://global.api.pvp.net/api/lol/static-data/{region}/v1.2/{rest}',
    'MATCHLIST_URL': 'matchlist/by-summoner/{summonerId}',
    'MATCH_URL': 'match/{matchId}',
    'CHAMPION_URL': 'champion/{id}?champData=all'
}

REGIONS = {
    'north america': 'na',
    'europe west': 'euw'
}

In [3]:
class Match(object):
    
    def __init__(self):
        self.winners = []
        self.losers = []
        self.duration = -1
        

In [4]:
class RiotAPI(object):
    
    def __init__(self, api_key, region=REGIONS['north america']):
        self.api_key = api_key
        self.region = region
        self.champions = {}
        self.champions_allinfo = {}
        self.champions_allinfo_saved = False
        
    def _request(self, base, rest, params={}):
        args = {'api_key': self.api_key}
        args.update(params)
        response = requests.get(
            URL[base].format(
                rest=rest,
                proxy=self.region,
                region=self.region,
            ),
            params=args
        )
        time.sleep(1.2)
        return response.json()
    
    def _base_request(self, rest, params={}):
        return self._request('BASE', rest, params)
        
    def _static_request(self, rest, params={}):
        return self._request('STATIC_BASE', rest, params)
    
    # my functions
    def _get_list_of_match_ids(self, sid):
        rest = URL['MATCHLIST_URL'].format(
            summonerId=sid
        )
        result = self._base_request(rest)
        if 'matches' in result:
            for match in result['matches']:
                yield match['matchId']
                
    def _get_match(self, mid):
        rest = URL['MATCH_URL'].format(
            matchId=mid
        )
        result = self._base_request(rest)
        return result
        
    def get_champion_all_info(self, championId):
        rest = URL['CHAMPION_URL'].format(
            id=championId
        )
        result = self._static_request(rest)
        return result
        
    def get_champion_index(self, championId):
        if championId not in self.champions:
            myid = len(self.champions)
            self.champions_allinfo[myid] = self.get_champion_all_info(championId)
            if not self.champions_allinfo_saved and len(self.champions_allinfo) == 132:
                with open('ChampionsAllInfo.pkl', 'wb') as output:
                    pk.dump(self.champions_allinfo, output)
                    self.champions_allinfo_saved = True
            self.champions[championId] = myid
        return self.champions[championId]
    
    def get_matches_champions_before_game(self, just_indices=True):
        #for sid in range(1,1000000):    #for sid in range(1000000,5000000):
        with open('summonerId_list.pkl', 'rb') as f:
            summnoners = pk.load(f)
        for sid in summnoners:
            matchids = self._get_list_of_match_ids(sid)
            for matchid in matchids:
                match = self._get_match(matchid)
                if 'participants' in match:
                    losers = []
                    winners = []
                    duration = match['matchDuration']
                    for member in match['participants']:
                        if member['stats']['winner']:
                            if just_indices:
                                winners += [self.get_champion_index(member['championId'])]
                            else:
                                winners += (self.get_champion_capabilities(member['championId']))
                        else:
                            if just_indices:
                                losers += [self.get_champion_index(member['championId'])]
                            else:
                                losers += (self.get_champion_capabilities(member['championId']))
                    yield winners + losers + [duration]
                    
                    
with open('../MyKey2.key', 'r') as key_file:
    KEY = key_file.readline().strip()
api = RiotAPI(KEY)

# Choose which type of data you want and run the according snippet

Getting the data about champions' capabilities before the game (for prediction)

In [ ]:
MEMORY_LIMIT = 10
data = []
with open('dataset_champion_capabilities.csv', 'w') as fp:
    a = csv.writer(fp, delimiter=',')
    dt = api.get_matches_champions_before_game(just_indices=False)
    for d in dt:
        data.append(d)
        if len(data) > MEMORY_LIMIT:
            a.writerows(data)
            data.clear()
    a.writerows(data)

Getting the data about champions' indices before the game (for prediction)

In [5]:
MEMORY_LIMIT = 100
data = []
c = 0
with open('DatasetChampionIndices.csv', 'w') as fp:
    a = csv.writer(fp, delimiter=',')
    dt = api.get_matches_champions_before_game()
    for d in dt:
        data.append(d)
        c += 1
        if len(data) > MEMORY_LIMIT:
            a.writerows(data)
            data.clear()
        if c >= 20001:
            break
    a.writerows(data)